{'fall': 0,'run': 1,'walk': 2,'cycle': 3,'lay': 4,'squat': 5,'mop': 6,'drink': 7,'sweep': 8,'brushing_teeth': 9,
    'cut': 10,'eat': 11,'folding_clothes': 12,'hang_out_clothes': 13,'ironing': 14,'open_door': 15,'open_fridge': 16,
    'sit': 17,'stand': 18,'use_computer': 19,'wash_dish': 20,'wash_face': 21,'wash_window': 22,'watch_tv': 23,
    'watering_flowers': 24,'write': 25,'wc': 26,'play_phone': 27,'switch': 28}

In [ ]:
first_label = 19
last_label = 7

import numpy as np
train_x = np.load('/home/intelligence/Robin/Dataset/train_x.npy',allow_pickle=True)
train_y = np.load('/home/intelligence/Robin/Dataset/train_y.npy',allow_pickle=True)
test_x = np.load('/home/intelligence/Robin/Dataset/test_x.npy',allow_pickle=True)
test_y = np.load('/home/intelligence/Robin/Dataset/test_y.npy',allow_pickle=True)

valid_train_index = np.where((train_y == last_label) | (train_y == first_label))
filter_label_train_x = train_x[valid_train_index]
filter_label_train_y = train_y[valid_train_index]

valid_test_index = np.where((test_y == last_label) | (test_y == first_label))
filter_label_test_x = test_x[valid_test_index]
filter_label_test_y = test_y[valid_test_index]

print("x_train: {}, x_test: {}, y_train: {}, y_test: {}, ".format(filter_label_train_x.shape, filter_label_test_x.shape,filter_label_train_y.shape,filter_label_test_y.shape))

In [ ]:
# 为防止标签越界，要将标签转化成0开始
y_train_convert, y_test_convert = [], []
for i in filter_label_train_y:
    if i==first_label:
        y_train_convert.append(0)
    elif i==last_label:
        y_train_convert.append(1)

for i in filter_label_test_y:
    if i==first_label:
        y_test_convert.append(0)
    elif i==last_label:
        y_test_convert.append(1)
y_train = np.array(y_train_convert)
y_test = np.array(y_test_convert)

**SVM作二分类**

In [ ]:
# from sklearn import svm
# from sklearn.model_selection import GridSearchCV
# classifier=svm.SVC()
# parameters=[{'kernel': ['rbf'], 'gamma': [0.01, 0.001, 0.0001, 0.00001], 'C': [1, 10, 100, 1000]}]
# model=GridSearchCV(classifier,parameters,n_jobs=-1,cv=4,verbose=1)
# model.fit(train_x,train_y)

In [ ]:
# from sklearn.metrics import accuracy_score
# ypred=model.predict(test_x)
# accuracy=accuracy_score(test_y,ypred)
# print('Best Parameters: '+ str(model.best_params_))
# print('Accuracy Score: '+ str(accuracy*100) + ' %')

**深度学习框架作二分类**

In [ ]:
import torch
import torch.nn as nn
from collections import OrderedDict

class FCN(nn.Module):
    def __init__(self, n_channels=16, n_timesteps=350, n_classes=2, out_channels=128):
        super(FCN, self).__init__()

        self.conv_block1 = nn.Sequential(nn.Conv1d(n_channels, 32, kernel_size=8, stride=1, bias=False, padding=4),
                                         nn.BatchNorm1d(32),
                                         nn.ReLU(),
                                         nn.MaxPool1d(kernel_size=2, stride=2, padding=1),
                                         nn.Dropout(0.35))
        self.conv_block2 = nn.Sequential(nn.Conv1d(32, 64, kernel_size=8, stride=1, bias=False, padding=4),
                                         nn.BatchNorm1d(64),
                                         nn.ReLU(),
                                         nn.MaxPool1d(kernel_size=2, stride=2, padding=1))
        self.conv_block3 = nn.Sequential(nn.Conv1d(64, out_channels, kernel_size=8, stride=1, bias=False, padding=4),
                                         nn.BatchNorm1d(out_channels),
                                         nn.ReLU(),
                                         nn.MaxPool1d(kernel_size=2, stride=2, padding=1))

        self.out_len = n_timesteps
        for _ in range(3):
            self.out_len = (self.out_len + 1) // 2 + 1

        self.out_channels = out_channels  # 128
        self.out_dim = self.out_len * self.out_channels

        self.logits = nn.Linear(self.out_len * out_channels, n_classes)

    def forward(self, x_in):
        x_in = x_in.permute(0, 2, 1)
        x = self.conv_block1(x_in)
        x = self.conv_block2(x)
        x = self.conv_block3(x)

        x_flat = x.reshape(x.shape[0], -1)
        logits = self.logits(x_flat)
        return logits


class TPN(nn.Module):
    def __init__(self, in_channels=16, n_classes=2):
        super().__init__()
        kernel_list = [24, 16, 8]
        channels_size = [32, 64, 96]

        self.out_dim = channels_size[2]
        convNet_layers = [
            ('conv1d_1', nn.Conv1d(in_channels, channels_size[0], kernel_size=(kernel_list[0],), stride=(1,))),
            ('relu1', nn.ReLU()),
            ('dropout1', nn.Dropout(p=0.1)),
            ('conv1d_2', nn.Conv1d(channels_size[0], channels_size[1], kernel_size=(kernel_list[1],), stride=(1,))),
            ('relu2', nn.ReLU()),
            ('dropout2', nn.Dropout(p=0.1)),
            ('conv1d_3', nn.Conv1d(channels_size[1], channels_size[2], kernel_size=(kernel_list[2],), stride=(1,))),
            ('relu3', nn.ReLU()),
            ('dropout3', nn.Dropout(p=0.1)),
            ('globalMaxPool', nn.AdaptiveMaxPool1d(output_size=1)),
            ('flatten', nn.Flatten()),
        ]
        self.convNet = nn.Sequential(OrderedDict(convNet_layers))
        self.classifier = nn.Linear(self.out_dim, n_classes)

    def forward(self, x, data_format='channel_last'):
        if data_format == 'channel_last':
            x = x.permute(0, 2, 1)
        x = self.convNet(x)
        out = self.classifier(x)
        return out


class DeepConvLSTM(nn.Module):
    def __init__(self, n_channels=16, n_classes=2, conv_kernels=64, kernel_size=5, LSTM_units=128):
        super(DeepConvLSTM, self).__init__()

        self.conv1 = nn.Conv2d(1, conv_kernels, (kernel_size, 1))
        self.conv2 = nn.Conv2d(conv_kernels, conv_kernels, (kernel_size, 1))
        self.conv3 = nn.Conv2d(conv_kernels, conv_kernels, (kernel_size, 1))
        self.conv4 = nn.Conv2d(conv_kernels, conv_kernels, (kernel_size, 1))

        self.dropout = nn.Dropout(0.5)
        self.lstm = nn.LSTM(n_channels * conv_kernels, LSTM_units, num_layers=1)

        self.out_dim = LSTM_units

        self.classifier = nn.Linear(LSTM_units, n_classes)

        self.activation = nn.ReLU()

    def forward(self, x):
        self.lstm.flatten_parameters()
        x = x.unsqueeze(1)
        x = self.activation(self.conv1(x))
        x = self.activation(self.conv2(x))
        x = self.activation(self.conv3(x))
        x = self.activation(self.conv4(x))

        x = x.permute(2, 0, 3, 1)
        x = x.reshape(x.shape[0], x.shape[1], -1)

        x = self.dropout(x)

        x, h = self.lstm(x)
        x = x[-1, :, :]

        out = self.classifier(x)
        return out

In [ ]:
import numpy as np
import random
import os
import time
from torch.utils import data
import sklearn.metrics
from d2l import torch as d2l

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def load_array(data_arrays, batch_size, is_train=True):
    """构造一个pytorch数据构造器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

def train(net, train_loader, test_loader, loss, num_epochs, optimizer, metrics):
    """训练模型"""
    # GPU训练
    train_on_gpu = torch.cuda.is_available()
    # train_on_gpu = False
    if(train_on_gpu):
        print('Training in GPU')
    else:
        print('No GPU available, training on CPU; consider making n_epochs vertargets small.')
    device = 'cuda' if train_on_gpu else 'cpu'
    print('training on', device)
    net.to(device)

    # 声明需要保存的量
    train_loss, test_loss, train_metrics, test_metrics, time_list = [], [], [], [], []
    max_test_metrics = 0.0
    # animator = d2l.Animator(xlabel='epoch', xlim=[1,num_epochs], ylim=[0,1],legend=['train loss', 'test loss', 'test accuracy'])
    for epoch in range(num_epochs):
        train_loss_batch, test_loss_batch, train_metrics_batch, test_metrics_batch = [], [], [], []
        start = time.time()
        # 训练
        if isinstance(net, nn.Module):
            net.train()  # 设置成训练模式，要计算梯度
        for X, y in train_loader:
            # 获取训练数据和标签
            X, y = X.to(device).float(), y.to(device).long()
            # 数据输入网络，前向传播
            y_hat = net(X)
            # 网络输出和标签输入损失函数，求得损失
            l = loss(y_hat, y)
            if isinstance(optimizer, torch.optim.Optimizer):
                optimizer.zero_grad()  # 清零优化器，梯度清零
                l.backward()  # 损失反向传播，计算梯度
                optimizer.step()  # 更新优化器，更新参数
            # 累加损失
            train_loss_batch.append(l.item())
            train_metrics_batch.append(metrics(y.cpu(), torch.argmax(y_hat, dim=1).cpu()))
        # 累加训练损失
        train_loss.append(np.mean(train_loss_batch))
        train_metrics.append(np.mean(train_metrics_batch))
        time_list.append(time.time() - start)
        # 验证
        if isinstance(net, nn.Module):  # 如果是用torch.nn实现的模型，就将它转换成评估模式
            net.eval()  # 将模型设置为评估模式：不再计算梯度，只做forward pass
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y.long())
            test_loss_batch.append(l.item())
            test_metrics_batch.append(metrics(y.cpu(), torch.argmax(y_hat, dim=1).cpu()))
        # 累加验证损失
        test_loss.append(np.mean(test_loss_batch))
        test_metrics.append(np.mean(test_metrics_batch))
        # 打印信息
        print("Time:{:.3f}s...".format(time_list[-1]),
              "Epoch:{}/{}...".format(epoch + 1, num_epochs),
              "Train Loss:{:.4f}...".format(train_loss[-1]),
              "Train Metrics:{:.4f}...".format(train_metrics[-1]),
              "Val Loss:{:.4f}...".format(test_loss[-1]),
              "Val Metrics:{:.4f}...".format(test_metrics[-1]),
              "Lr:{:.5f}...".format(optimizer.state_dict()['param_groups'][0]['lr']))
        if max_test_metrics < test_metrics[-1] and epoch > num_epochs*0.7:
            max_test_metrics = test_metrics[-1]

#         animator.add(epoch+1, (train_loss[-1],test_loss[-1],test_metrics[-1]))
    print("max_test_metrics:{:.4f}...".format(max_test_metrics))
    return train_loss, test_loss, train_metrics, test_metrics

In [ ]:
set_seed(2023)
train_x = torch.Tensor(filter_label_train_x)
train_y = torch.Tensor(y_train)
test_x = torch.Tensor(filter_label_test_x)
test_y = torch.Tensor(y_test)
batch_size = 64
train_loader = load_array((train_x, train_y), batch_size)
test_loader = load_array((test_x, test_y), batch_size)
loss = nn.CrossEntropyLoss()
# net = DeepConvLSTM()
net = FCN()
# net = TPN()
# # optimizer = torch.optim.SGD(net.parameters(), lr=0.03, weight_decay=1e-4)    # DeepConvLSTM
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)   # DeepConvLSTM
num_epochs = 100
train_loss, test_loss, train_metrics, test_metrics = \
    train(net, train_loader, test_loader, loss, num_epochs, optimizer, sklearn.metrics.accuracy_score)